In [1]:
import pandas as pd
import glob
import tqdm


In [2]:
# Preprocess text (username and link placeholders)
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)


In [7]:
glob.glob('raw_data/CES_*')

['raw_data\\CES_tweets_2018_fully_cleaned_with_dates.csv',
 'raw_data\\CES_tweets_2019_fully_cleaned_with_dates.csv',
 'raw_data\\CES_tweets_2020_fully_cleaned_with_dates.csv',
 'raw_data\\CES_tweets_2021_fully_cleaned_with_dates.csv',
 'raw_data\\CES_tweets_2022_fully_cleaned_with_dates.csv']

In [5]:
file_list = glob.glob('..//raw_data/CES_*')
pre_processed_dataframes = []
year = [2018,2019,2020,2021,2022]

In [ ]:
for idx, file in enumerate(tqdm.tqdm(file_list[:-1])): # Excludes the 2022 due to divergent format explained below in next block.
    df = pd.read_csv(file)
    df = df[['id', 'date', 'text']]
    df['text'] = df['text'].map(preprocess)
    df = df.drop_duplicates(subset=['text'])
    df.to_csv(f'preprocessed_year_data/processed_{year[idx]}.csv')
    pre_processed_dataframes.append(df)


In [6]:
#The 2022 does not include a pure date column and as such one has to be mapped from the datetime
df = pd.read_csv(file_list[-1])
df = df[['id', 'datetime', 'text']]
df = df.rename(columns={'datetime':'date'})
df['date'] = pd.to_datetime(df['date']).dt.date
df['text'] = df['text'].map(preprocess)
df = df.drop_duplicates(subset=['text'])
df.to_csv(f'preprocessed_year_data/processed_{year[-1]}.csv')
pre_processed_dataframes.append(df)

In [13]:
# If local machine cannot handle the initial RAM load of loading all 5 year .csv files for load,
# a post load of the processed files can be done to then collect them into a list as the previous block also attempted:
pre_processed_dataframes = []
for file in glob.glob('preprocessed_year_data/processed_*.csv'):
    pre_processed_dataframes.append(pd.read_csv(file,index_col=0))


In [15]:
# Concatenates all 5 dataframes into 1 and exports it to its own file
pd.concat(pre_processed_dataframes).to_csv('preprocessed_year_data/collected_processed.csv')